In [7]:
import string
import math
import getpass # Use getpass for secure password entry (doesn't display input)

def password_strength_meter(password: str) -> dict:

    score = 0
    feedback = []
    
    # Define character sets for variety check and entropy pool
    LOWER = string.ascii_lowercase
    UPPER = string.ascii_uppercase
    DIGITS = string.digits
    SYMBOLS = string.punctuation
    

    COMMON_PASSWORDS = {"password", "12345678", "qwerty", "admin", "P@ssword"}
    
    # Check against lowercase version for case-insensitivity against common words
    if password.lower() in COMMON_PASSWORDS:
        score -= 5
        feedback.append("⚠️ Very common, easily guessed password.")
    

    length = len(password)
    
    if length >= 8:
        score += 2
        feedback.append(" Meets 8-character minimum.")
    if length >= 12:
        score += 2
        feedback.append(" Meets 12-character minimum.")
        

    
    # Track variety presence and pool size for entropy calculation
    char_checks = {
        "lowercase": any(c in LOWER for c in password),
        "uppercase": any(c in UPPER for c in password),
        "digit": any(c in DIGITS for c in password),
        "symbol": any(c in SYMBOLS for c in password),
    }
    
    for char_type, is_present in char_checks.items():
        if is_present:
            score += 1
            feedback.append(f" Contains a {char_type} letter/character.")
        else:
            feedback.append(f" Missing a {char_type} letter/character.")

    # Cap the score between 0 and 10
    final_score = max(0, min(10, score))
    
    if final_score >= 8:
        strength_rating = " Very Strong"
    elif final_score >= 6:
        strength_rating = " Strong"
    elif final_score >= 4:
        strength_rating = " Moderate"
    else:
        strength_rating = " Weak"


    # Calculate pool size based on present character types
    pool_size = 0
    if char_checks["lowercase"]:
        pool_size += len(LOWER)
    if char_checks["uppercase"]:
        pool_size += len(UPPER)
    if char_checks["digit"]:
        pool_size += len(DIGITS)
    if char_checks["symbol"]:
        pool_size += len(SYMBOLS)
        
    # Approximate entropy: length * log2(pool_size)
    approx_entropy = 0
    if pool_size > 0 and length > 0:
        approx_entropy = length * math.log2(pool_size)
    
    # --- Format Results ---
    return {
        "score": final_score,
        "rating": strength_rating,
        "length": length,
        "pool_size": pool_size,
        "approx_entropy_bits": round(approx_entropy, 2),
        "feedback": feedback,
    }


if __name__ == "__main__":
    print("\n---  Password Strength Meter ---")
    
    # Use getpass to securely read the password without echoing it to the console
    try:
        user_password = getpass.getpass("Enter the password to check (input hidden): ")
    except Exception as e:
        # Fallback if getpass isn't available (e.g., in some IDEs/notebooks)
        print(f"\ngetpass failed: {e}. Falling back to standard input.")
        user_password = input("Enter the password to check (input visible): ")
        
    if not user_password:
        print("\n No password entered. Exiting.")
    else:
        # Run the meter function
        result = password_strength_meter(user_password)
        
        print("\n" + "="*40)
        print(" STRENGTH REPORT")
        print("="*40)
        
        # Print results clearly
        print(f"**Password Length:** {result['length']}")
        print(f"**Character Pool Size (Total Options):** {result['pool_size']}")
        print(f"**Approximate Entropy:** {result['approx_entropy_bits']} bits")
        print(f"**Calculated Score (Max 10):** {result['score']}")
        print(f"**Overall Rating:** {result['rating']}\n")
        
        print("--- Detailed Feedback ---")
        for line in result['feedback']:
            print(f"- {line}")
        print("-------------------------\n")


---  Password Strength Meter ---

 STRENGTH REPORT
**Password Length:** 11
**Character Pool Size (Total Options):** 94
**Approximate Entropy:** 72.1 bits
**Calculated Score (Max 10):** 6
**Overall Rating:**  Strong

--- Detailed Feedback ---
-  Meets 8-character minimum.
-  Contains a lowercase letter/character.
-  Contains a uppercase letter/character.
-  Contains a digit letter/character.
-  Contains a symbol letter/character.
-------------------------



In [2]:
import hashlib
import bcrypt
import os
import time

# --- Configuration ---
PASSWORD = "my_secret_password_123"
PASSWORD_BYTES = PASSWORD.encode('utf-8')

print(f"--- Testing Hashing Functions ---")
print(f"Original Password: '{PASSWORD}'")
print("-" * 35)



def hash_standard(password_bytes: bytes, algorithm: str) -> str:
    """Hashes bytes using a standard algorithm (MD5 or SHA256)."""
    h = hashlib.new(algorithm)
    h.update(password_bytes)
    return h.hexdigest()

print("## Standard Hashing (hashlib)")

# MD5 Test
md5_hash = hash_standard(PASSWORD_BYTES, 'md5')
print(f"MD5 Hash (32 chars):    {md5_hash}")

# SHA256 Test
sha256_hash = hash_standard(PASSWORD_BYTES, 'sha256')
print(f"SHA256 Hash (64 chars): {sha256_hash}")

# Demonstrating why they are weak: they are too fast and fixed
start_time = time.perf_counter()
for _ in range(1000):
    hash_standard(PASSWORD_BYTES, 'sha256')
end_time = time.perf_counter()
print(f"\nSHA256 time (1000 iterations): {round((end_time - start_time) * 1000, 4)} ms")
print("-" * 35)


def hash_bcrypt(password_bytes: bytes) -> bytes:
    """Generates a random salt and hashes the password using bcrypt."""
    # bcrypt.gensalt() generates the random salt and sets the work factor (cost)
    # The salt is automatically embedded into the resulting hash string.
    start_time = time.perf_counter()
    hashed_password = bcrypt.hashpw(password_bytes, bcrypt.gensalt())
    end_time = time.perf_counter()
    print(f"\nGeneration time: {round((end_time - start_time) * 1000, 4)} ms (Deliberately slow!)")
    return hashed_password

def verify_bcrypt(password_attempt_bytes: bytes, stored_hash_bytes: bytes) -> bool:
    """Verifies a password attempt against a stored bcrypt hash."""
    # bcrypt extracts the salt from the stored hash and uses it for verification.
    return bcrypt.checkpw(password_attempt_bytes, stored_hash_bytes)

print("## bcrypt Hashing (Slow and Salted)")

# --- 1. Hashing (Registration) ---
print("\n--- 1. Hashing (Simulated Registration) ---")
# The resulting hash includes the algorithm, cost, and salt (e.g., $2b$12$...)
stored_hash = hash_bcrypt(PASSWORD_BYTES)
print(f"Stored Hash (includes salt): {stored_hash.decode('utf-8')}")

# --- 2. Verification (Login Attempt) ---
print("\n--- 2. Verification (Simulated Login) ---")

# Correct Password Attempt
is_correct = verify_bcrypt(PASSWORD_BYTES, stored_hash)
print(f"Attempt 1: Correct Password ('{PASSWORD}') -> Match: {is_correct}")

# Incorrect Password Attempt
INCORRECT_PASSWORD = "wrong_password_123".encode('utf-8')
is_incorrect = verify_bcrypt(INCORRECT_PASSWORD, stored_hash)
print(f"Attempt 2: Incorrect Password -> Match: {is_incorrect}")

--- Testing Hashing Functions ---
Original Password: 'my_secret_password_123'
-----------------------------------
## Standard Hashing (hashlib)
MD5 Hash (32 chars):    148e10bc860b493088a576534860bc47
SHA256 Hash (64 chars): 2b92cdf061d87c33e5e2da1a4561e722beb3bc2eb1addbde8accc93e37c32518

SHA256 time (1000 iterations): 0.7383 ms
-----------------------------------
## bcrypt Hashing (Slow and Salted)

--- 1. Hashing (Simulated Registration) ---

Generation time: 181.9825 ms (Deliberately slow!)
Stored Hash (includes salt): $2b$12$jbiTp3.lMF7p5P/pt5HDyO23hnPhDWwq/.RqesU5fKKieZI1WYv2.

--- 2. Verification (Simulated Login) ---
Attempt 1: Correct Password ('my_secret_password_123') -> Match: True
Attempt 2: Incorrect Password -> Match: False


In [3]:
import hashlib
import os
import binascii

# --- Configuration ---
PASSWORD = "user123password"
PASSWORD_BYTES = PASSWORD.encode('utf-8')
SECRET_PEPPER = os.urandom(32) # A secret key stored outside the database

print("--- Password Hashing Demonstration ---")
print(f"Password to hash: '{PASSWORD}'")
print("-" * 50)



def hash_unsalted(password_bytes: bytes) -> str:
    """Hashes a password without salt."""
    return hashlib.sha256(password_bytes).hexdigest()

print("1. HASHING WITHOUT SALT (Vulnerable)")
hash1_unsalted = hash_unsalted(PASSWORD_BYTES)
hash2_unsalted = hash_unsalted(PASSWORD_BYTES)

print(f"Attempt 1 Hash: {hash1_unsalted}")
print(f"Attempt 2 Hash: {hash2_unsalted}")
if hash1_unsalted == hash2_unsalted:
    print(" Result: Hashes are IDENTICAL. This is vulnerable to pre-computed attacks (Rainbow Tables).")
print("-" * 50)



def hash_with_salt(password_bytes: bytes, salt: bytes) -> str:
    """Hashes a password combined with a unique, user-specific salt."""
    # Salt is prepended to the password before hashing
    return hashlib.sha256(salt + password_bytes).hexdigest()

print("2. HASHING WITH SALT (Defeats Rainbow Tables)")

# --- Attempt 1 (Simulated Registration) ---
salt1 = os.urandom(16) # Generate a unique random salt for user 1
hash1_salted = hash_with_salt(PASSWORD_BYTES, salt1)
print(f"Salt 1 (Hex):   {binascii.hexlify(salt1).decode()}")
print(f"Hash 1 (Stored): {hash1_salted}")

# --- Attempt 2 (Simulated Registration of Same Password by different User) ---
salt2 = os.urandom(16) # Generate a NEW, unique random salt for user 2
hash2_salted = hash_with_salt(PASSWORD_BYTES, salt2)
print(f"Salt 2 (Hex):   {binascii.hexlify(salt2).decode()}")
print(f"Hash 2 (Stored): {hash2_salted}")

if hash1_salted != hash2_salted:
    print(" Result: Hashes are DIFFERENT. A Rainbow Table attack fails because the pre-computed hash for the password no longer matches.")
print("-" * 50)



def hash_with_pepper(password_bytes: bytes, salt: bytes, pepper: bytes) -> str:
    """Hashes a password using both a unique salt and a secret pepper."""
    # Pepper is a secret key known only to the server. 
    # Hashing order: Pepper + Salt + Password
    return hashlib.sha256(pepper + salt + password_bytes).hexdigest()

print("3. HASHING WITH SALT AND PEPPER (Protects against Database Theft)")

# --- Simulated Storage ---
final_salt = os.urandom(16)
final_hash = hash_with_pepper(PASSWORD_BYTES, final_salt, SECRET_PEPPER)
# Database stores: final_salt (visible) and final_hash (visible)
# Server stores: SECRET_PEPPER (kept secret)

print(f"Secret Pepper (Hex):   {binascii.hexlify(SECRET_PEPPER).decode()}")
print(f"Unique Salt (Hex):     {binascii.hexlify(final_salt).decode()}")
print(f"Final Hash (Stored):   {final_hash}")

# --- Simulated Login Verification ---
# The attacker would steal the database and have the salt and hash, but NOT the pepper.
# To verify, the server uses all three components:
ATTEMPT_PASSWORD_BYTES = "user123password".encode('utf-8')

# The verification process needs the stored salt and the secret pepper
verification_hash = hash_with_pepper(ATTEMPT_PASSWORD_BYTES, final_salt, SECRET_PEPPER)

print(f"\nVerification Hash:     {verification_hash}")

if verification_hash == final_hash:
    print(" Verification Success: The server confirmed the password using the salt and secret pepper.")
else:
    print(" Verification Failure")
print("-" * 50)

--- Password Hashing Demonstration ---
Password to hash: 'user123password'
--------------------------------------------------
1. HASHING WITHOUT SALT (Vulnerable)
Attempt 1 Hash: 6384e02d9a47ac61c64950915c878ea54cc91a3d9c11d7497818017db8356d2a
Attempt 2 Hash: 6384e02d9a47ac61c64950915c878ea54cc91a3d9c11d7497818017db8356d2a
 Result: Hashes are IDENTICAL. This is vulnerable to pre-computed attacks (Rainbow Tables).
--------------------------------------------------
2. HASHING WITH SALT (Defeats Rainbow Tables)
Salt 1 (Hex):   cd6d9be13229a43faa6e5a9a273a87f7
Hash 1 (Stored): e3c978847824b8b293f7c515bf894c82c825b151d3e0e335603a600bf5327943
Salt 2 (Hex):   f73b4d148e4b9a75730918b245cad75c
Hash 2 (Stored): c3b64e441c77e59e9239644e3b82fcf42c83b7a5ecad27f8175ba5afd786b41c
 Result: Hashes are DIFFERENT. A Rainbow Table attack fails because the pre-computed hash for the password no longer matches.
--------------------------------------------------
3. HASHING WITH SALT AND PEPPER (Protects again

In [4]:
import pyotp
import time
import qrcode
import sys

# --- Configuration ---
# 1. Generate a random base32 secret (this is what you'd store per user in a real database)
# pyotp generates strong secrets. It must be base32 encoded.
SECRET_KEY = pyotp.random_base32()
ISSUER_NAME = "MySecureApp"
USER_ACCOUNT = "user@example.com"

def generate_provisioning_uri(secret: str, account_name: str, issuer: str) -> str:
    """Generates the URI string that the authenticator app uses for setup."""
    # pyotp.TOTP is configured with the secret
    totp = pyotp.TOTP(secret)
    # Get the provisioning URI in the format compatible with authenticator apps (OTPAuth)
    return totp.provisioning_uri(
        name=account_name,
        issuer_name=issuer
    )

def generate_and_display_qr(uri: str):
    """Generates and displays a QR code for the user to scan."""
    try:
        # Create QR code image
        img = qrcode.make(uri)
        # Save or display the image (for demonstration, we save it)
        filename = "totp_qrcode.png"
        img.save(filename)
        print(f" QR Code generated and saved as '{filename}'. Scan this with your 2FA app.")
    except Exception as e:
        # qrcode might not work in all environments (like some terminals/notebooks)
        print(f" Could not generate QR code image (Install 'qrcode' or run in a suitable environment). Error: {e}")
        
def demo_totp_authenticator():
    """Demonstrates TOTP setup, generation, and verification."""
    
    print("---  TOTPAuthenticator Demo ---")
    print(f"Generated Secret Key: {SECRET_KEY}")
    print("-" * 30)


    
    uri = generate_provisioning_uri(SECRET_KEY, USER_ACCOUNT, ISSUER_NAME)
    print("STEP 1: Provisioning")
    print(f"Provisioning URI (OTPAuth link): {uri}")
    generate_and_display_qr(uri)

    # Re-instantiate the TOTP object using the stored secret key
    totp = pyotp.TOTP(SECRET_KEY)
    
    # ------------------------------------------------------------------
    # 2. Server-side Generation (For reference/debug)
    # ------------------------------------------------------------------
    
    print("\nSTEP 2: Server-Side Code Generation (Current Time)")
    # The code is valid for a 30-second window by default
    current_code = totp.now() 
    print(f"Current valid TOTP code: {current_code} (Expires in ~{totp.interval} seconds)")
    
    # ------------------------------------------------------------------
    # 3. Verification Phase: User provides the code
    # ------------------------------------------------------------------
    
    print("\nSTEP 3: User Verification")
    print(" Please open your Authenticator app (e.g., Google Authenticator) and scan the QR code now.")
    
    # Wait for a moment to allow time for the user to scan and for the code interval to reset
    time.sleep(3) 

    try:
        # Use getpass to securely read the input without displaying it
        import getpass
        user_input = getpass.getpass("Enter the current 6-digit TOTP code from your app: ")
    except Exception:
        # Fallback for environments where getpass fails
        user_input = input("Enter the current 6-digit TOTP code from your app: ")

    # Check the user's code. The 'verify' method checks the current window AND the 
    # preceding/succeeding windows (time drift) for robustness.
    is_valid = totp.verify(user_input, valid_window=1) 

    print("\n--- Verification Result ---")
    if is_valid:
        print(" SUCCESS: The code is valid! User logged in.")
    else:
        print(" FAILURE: Invalid code. Check the code and ensure your phone time is accurate.")
    
if __name__ == "__main__":
    demo_totp_authenticator()

---  TOTPAuthenticator Demo ---
Generated Secret Key: ASTNC4QRASV6LZ6R5NF75IC4XZ67FF7B
------------------------------
STEP 1: Provisioning
Provisioning URI (OTPAuth link): otpauth://totp/MySecureApp:user%40example.com?secret=ASTNC4QRASV6LZ6R5NF75IC4XZ67FF7B&issuer=MySecureApp
 QR Code generated and saved as 'totp_qrcode.png'. Scan this with your 2FA app.

STEP 2: Server-Side Code Generation (Current Time)
Current valid TOTP code: 006063 (Expires in ~30 seconds)

STEP 3: User Verification
 Please open your Authenticator app (e.g., Google Authenticator) and scan the QR code now.

--- Verification Result ---
 FAILURE: Invalid code. Check the code and ensure your phone time is accurate.


In [5]:
import hashlib
import time

def simulate_dictionary_attack(target_hash: str, hash_type: str) -> None:
    common_passwords = [
        "password", "123456", "qwerty", "secret", "user123", 
        "iloveyou", "access", "admin", "p@ssword", "dragon"
    ]
    
    # Standardize the hash type input
    hash_type = hash_type.lower()
    
    print(f" Starting Dictionary Attack ({hash_type.upper()}) ---")
    print(f"Target Hash: {target_hash}")
    
    start_time = time.time()
    found = False
    
    for i, attempt in enumerate(common_passwords):
        # Hash the attempted password using the specified algorithm
        
        # 4. Hash the common password
        h = hashlib.new(hash_type)
        h.update(attempt.encode('utf-8'))
        attempt_hash = h.hexdigest()
        
        print(f"  Attempt {i+1}: Testing '{attempt}' -> Hash: {attempt_hash[:10]}...") # Show a snippet of the hash
        
        # 5. Compare the result to the target_hash
        if attempt_hash == target_hash:
            end_time = time.time()
            elapsed = (end_time - start_time) * 1000 # Convert to milliseconds
            
            # 6. Report success
            print("\n==============================================")
            print(f"SUCCESS! Password found!")
            print(f"Original Password: **{attempt}**")
            print(f"Hashes checked: {i + 1}")
            print(f"Time Elapsed: **{round(elapsed, 4)} milliseconds**")
            print("==============================================")
            found = True
            break # Stop the loop once a match is found
            
    if not found:
        end_time = time.time()
        elapsed = (end_time - start_time) * 1000
        print(f"\nFAILED: Password not found in the dictionary list after checking {len(common_passwords)} passwords.")
        print(f"Total time taken: {round(elapsed, 4)} milliseconds.")


# Password 1: "password"
PASSWORD_1 = "password"
TARGET_HASH_MD5 = hashlib.md5(PASSWORD_1.encode('utf-8')).hexdigest()

# Password 2: "secure_pass" (Not in the common list)
PASSWORD_2 = "secure_pass"
TARGET_HASH_SHA256 = hashlib.sha256(PASSWORD_2.encode('utf-8')).hexdigest()

# Simulation 1: MD5 Crack (Quick success expected)
simulate_dictionary_attack(TARGET_HASH_MD5, 'md5')

print("\n" + "="*50 + "\n")

# Simulation 2: SHA-256 Failure (Password not in the dictionary)
simulate_dictionary_attack(TARGET_HASH_SHA256, 'sha256')

 Starting Dictionary Attack (MD5) ---
Target Hash: 5f4dcc3b5aa765d61d8327deb882cf99
  Attempt 1: Testing 'password' -> Hash: 5f4dcc3b5a...

SUCCESS! Password found!
Original Password: **password**
Hashes checked: 1
Time Elapsed: **0.0141 milliseconds**


 Starting Dictionary Attack (SHA256) ---
Target Hash: 5bd2e6e6283fda1a15d33befdd4584fa0e33923b07c2cebd8f9fcd62944872a7
  Attempt 1: Testing 'password' -> Hash: 5e884898da...
  Attempt 2: Testing '123456' -> Hash: 8d969eef6e...
  Attempt 3: Testing 'qwerty' -> Hash: 65e84be335...
  Attempt 4: Testing 'secret' -> Hash: 2bb80d537b...
  Attempt 5: Testing 'user123' -> Hash: e606e38b0d...
  Attempt 6: Testing 'iloveyou' -> Hash: e4ad93ca07...
  Attempt 7: Testing 'access' -> Hash: a0561fd649...
  Attempt 8: Testing 'admin' -> Hash: 8c6976e5b5...
  Attempt 9: Testing 'p@ssword' -> Hash: 0fd205965c...
  Attempt 10: Testing 'dragon' -> Hash: a9c43be948...

FAILED: Password not found in the dictionary list after checking 10 passwords.
Total tim

In [6]:
import bcrypt
import pyotp
import string
import math
import time
import getpass
import os

# --- 1. MOCK DATABASE ---
# Stores {username: {'hash': bytes, 'totp_secret': str}}
USERS_DB = {}

# --- 2. SECURITY UTILITIES ---

def is_password_strong_enough(password: str) -> bool:
    """
    Analyzes password strength based on length and character variety.
    Returns True if score >= 6, False otherwise.
    """
    score = 0
    length = len(password)
    
    # Length Check (Minimum 8 chars)
    if length >= 8:
        score += 3
        
    # Character Variety Check
    has_upper = any(c in string.ascii_uppercase for c in password)
    has_lower = any(c in string.ascii_lowercase for c in password)
    has_digit = any(c in string.digits for c in password)
    has_symbol = any(c in string.punctuation for c in password)
    
    if has_upper: score += 1
    if has_lower: score += 1
    if has_digit: score += 1
    if has_symbol: score += 1
    
    # For this demo, require a score of 6 out of a possible 7
    # (i.e., must be 8+ chars and have at least 3 variety types)
    return score >= 6

def hash_password(password_bytes: bytes) -> bytes:
    """Hashes the password using bcrypt, which automatically handles salting."""
    # bcrypt.gensalt() generates a unique random salt and sets the work factor
    return bcrypt.hashpw(password_bytes, bcrypt.gensalt())

def verify_password(password_attempt_bytes: bytes, stored_hash_bytes: bytes) -> bool:
    """Verifies a password attempt against the stored bcrypt hash."""
    # bcrypt extracts the salt from the stored_hash_bytes and uses it for verification
    return bcrypt.checkpw(password_attempt_bytes, stored_hash_bytes)


def register_user(username: str, password: str) -> bool:
    """
    Registers a new user after checking password strength and setting up TOTP.
    """
    if username in USERS_DB:
        print(f" User '{username}' already exists.")
        return False
        
    # 1. Analyze password strength
    if not is_password_strong_enough(password):
        print(" Registration failed: Password is too weak.")
        print("   Required: 8+ chars, mix of upper/lower/digits/symbols.")
        return False
        
    password_bytes = password.encode('utf-8')
    
    # 2. Securely hash the password (with automatic salt)
    password_hash = hash_password(password_bytes)
    
    # 3. Create a TOTP Authenticator secret
    totp_secret = pyotp.random_base32()
    
    # 4. Store hash and secret
    USERS_DB[username] = {
        'hash': password_hash,
        'totp_secret': totp_secret
    }
    
    # Generate the provisioning URI for the user to set up 2FA
    uri = pyotp.totp.TOTP(totp_secret).provisioning_uri(
        name=username,
        issuer_name="SecureUserMgmt"
    )

    print(f"\n User '{username}' registered successfully!")
    print("   --- 2FA Setup Required ---")
    print(f"   TOTP Secret Key (for 2FA app): {totp_secret}")
    print(f"   Scan this URI: {uri}")
    print("--------------------------------")
    return True

def authenticate_user(username: str, password_attempt: str, totp_code: str) -> bool:
    """
    Authenticates a user by verifying password hash and TOTP code.
    """
    if username not in USERS_DB:
        print(" Authentication failed: Invalid username.")
        return False
        
    user_data = USERS_DB[username]
    password_attempt_bytes = password_attempt.encode('utf-8')

    # 1. Verify the password (securely, using salt extracted from hash)
    if not verify_password(password_attempt_bytes, user_data['hash']):
        print(" Authentication failed: Invalid password.")
        return False

    # 2. Verify the TOTP code
    totp = pyotp.TOTP(user_data['totp_secret'])
    
    # pyotp.verify checks the current time window and one preceding/succeeding window (for drift)
    if not totp.verify(totp_code, valid_window=1):
        print(" Authentication failed: Invalid TOTP code.")
        print(f"   (Current server code: {totp.now()})")
        return False
        
    print(f"\n🎉 SUCCESS: User '{username}' fully authenticated.")
    return True


if __name__ == "__main__":
    
    # --- Part A: Registration Simulation ---
    print("## Part A: Registration")
    
    # 1. Reject weak password
    register_user("weak_user", "pass123") 
    
    # 2. Register strong user
    STRONG_PASS = "MyS3cur3P@ssw0rd!"
    register_user("testuser", STRONG_PASS)
    
    # 3. Simulate a successful setup in the user's 2FA app
    # We must instantiate a TOTP object to get the valid code for the login attempt.
    if "testuser" in USERS_DB:
        print("\n--- Simulating 2FA Setup ---")
        user_secret = USERS_DB['testuser']['totp_secret']
        test_totp = pyotp.TOTP(user_secret)
        
        # Give a moment for the user to 'type' the code
        time.sleep(1) 
        
        VALID_TOTP_CODE = test_totp.now()
        print(f"Simulated TOTP App Code: {VALID_TOTP_CODE}")

        print("\n" + "="*50)
        
        # --- Part B: Authentication Simulation ---
        print("## Part B: Authentication")
        
        # 1. Successful Login
        print("\n--- Successful Login Attempt ---")
        authenticate_user("testuser", STRONG_PASS, VALID_TOTP_CODE)
        
        # 2. Failed Password Attempt
        print("\n--- Failed Password Attempt ---")
        authenticate_user("testuser", "wrong_password", VALID_TOTP_CODE)
        
        # 3. Failed TOTP Attempt
        print("\n--- Failed TOTP Attempt ---")
        authenticate_user("testuser", STRONG_PASS, "000000") # Incorrect TOTP code

## Part A: Registration
 Registration failed: Password is too weak.
   Required: 8+ chars, mix of upper/lower/digits/symbols.

 User 'testuser' registered successfully!
   --- 2FA Setup Required ---
   TOTP Secret Key (for 2FA app): GLKIZWTG4IAWUZV52GJ3ZORG23KZSA6P
   Scan this URI: otpauth://totp/SecureUserMgmt:testuser?secret=GLKIZWTG4IAWUZV52GJ3ZORG23KZSA6P&issuer=SecureUserMgmt
--------------------------------

--- Simulating 2FA Setup ---
Simulated TOTP App Code: 535685

## Part B: Authentication

--- Successful Login Attempt ---

🎉 SUCCESS: User 'testuser' fully authenticated.

--- Failed Password Attempt ---
 Authentication failed: Invalid password.

--- Failed TOTP Attempt ---
 Authentication failed: Invalid TOTP code.
   (Current server code: 535685)
